In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/the-oxfordiiit-pet-dataset/images/american_pit_bull_terrier_16.jpg
/kaggle/input/the-oxfordiiit-pet-dataset/images/leonberger_200.jpg
/kaggle/input/the-oxfordiiit-pet-dataset/images/english_cocker_spaniel_35.jpg
/kaggle/input/the-oxfordiiit-pet-dataset/images/Siamese_67.jpg
/kaggle/input/the-oxfordiiit-pet-dataset/images/boxer_77.jpg
/kaggle/input/the-oxfordiiit-pet-dataset/images/saint_bernard_146.jpg
/kaggle/input/the-oxfordiiit-pet-dataset/images/american_bulldog_37.jpg
/kaggle/input/the-oxfordiiit-pet-dataset/images/Sphynx_239.jpg
/kaggle/input/the-oxfordiiit-pet-dataset/images/Egyptian_Mau_164.jpg
/kaggle/input/the-oxfordiiit-pet-dataset/images/Birman_116.jpg
/kaggle/input/the-oxfordiiit-pet-dataset/images/english_setter_178.jpg
/kaggle/input/the-oxfordiiit-pet-dataset/images/Siamese_110.jpg
/kaggle/input/the-oxfordiiit-pet-dataset/images/newfoundland_39.jpg
/kaggle/input/the-oxfordiiit-pet-dataset/images/pug_84.jpg
/kaggle/input/the-oxfordiiit-pet-dataset/images/yor

In [13]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers, models
import tf_keras 
import os
import pandas as pd

In [10]:
IMG_DIR = "/kaggle/input/the-oxfordiiit-pet-dataset/images"

In [14]:
import tensorflow as tf
import tensorflow_hub as hub
import os
import pandas as pd




all_images = [f for f in os.listdir(IMG_DIR) if f.endswith(".jpg")]

labels = [img.split("_")[0] for img in all_images]


df = pd.DataFrame({"file": all_images, "label_name": labels})


label_to_index = {name: idx for idx, name in enumerate(sorted(df["label_name"].unique()))}
df["label"] = df["label_name"].map(label_to_index)

num_classes = len(label_to_index)
print("Number of classes:", num_classes)


df["file"] = df["file"].apply(lambda x: os.path.join(IMG_DIR, x))


from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df["label"], random_state=42)


Number of classes: 35


In [16]:
def load_image(image_path, label):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, (224, 224))
    image = (tf.cast(image, tf.float32) / 127.5) - 1.0  # Scale to [-1, 1]
    return image, label

# Convert to tf.data.Dataset
train_ds = tf.data.Dataset.from_tensor_slices((train_df["file"].values, train_df["label"].values))
test_ds = tf.data.Dataset.from_tensor_slices((test_df["file"].values, test_df["label"].values))

train_ds = (
    train_ds.map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
    .shuffle(1000)
    .batch(32)
    .prefetch(tf.data.AUTOTUNE)
)

test_ds = (
    test_ds.map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(32)
    .prefetch(tf.data.AUTOTUNE)
)


I0000 00:00:1761103286.722878      37 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1761103286.723601      37 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


In [17]:


vit_url = "https://www.kaggle.com/models/spsayakpaul/vision-transformer/TensorFlow2/vit-b16-classification/1"

base_model = hub.KerasLayer(vit_url, trainable=True, input_shape=(224, 224, 3))


In [18]:

model = tf_keras.Sequential([
    tf_keras.layers.InputLayer(input_shape=(224, 224, 3)),
    base_model,
    tf_keras.layers.Dropout(rate=0.2),
    tf_keras.layers.Dense(num_classes, activation="softmax")
])

model.build((None,) + (224, 224, 3))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1000)              86567656  
                                                                 
 dropout (Dropout)           (None, 1000)              0         
                                                                 
 dense (Dense)               (None, 35)                35035     
                                                                 
Total params: 86602691 (330.36 MB)
Trainable params: 35035 (136.86 KB)
Non-trainable params: 86567656 (330.23 MB)
_________________________________________________________________


In [19]:
model.compile(
    optimizer=tf_keras.optimizers.Adam(learning_rate=1e-5),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)


In [20]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
early_stop = EarlyStopping(
    monitor='val_loss',   
    patience=7,           
    restore_best_weights=True  
)

checkpoint = ModelCheckpoint(
    'best_model.h5',      
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)

In [ ]:
history = model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=5,
   
)


Epoch 1/5


I0000 00:00:1761103329.836410     112 service.cc:148] XLA service 0x7e0b18059700 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1761103329.837350     112 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1761103329.837381     112 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1761103330.742533     112 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1761103333.749094     112 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


 14/185 [=>............................] - ETA: 1:00 - loss: 6.3781 - accuracy: 0.0268

Corrupt JPEG data: 240 extraneous bytes before marker 0xd9


 53/185 [=======>......................] - ETA: 47s - loss: 6.2574 - accuracy: 0.0383

...

185/185 [==============================] - 107s 573ms/step - loss: 4.9417 - accuracy: 0.0641 - val_loss: 4.0259 - val_accuracy: 0.0866
Epoch 3/5
 14/185 [=>............................] - ETA: 1:17 - loss: 4.4606 - accuracy: 0.1094

Corrupt JPEG data: 240 extraneous bytes before marker 0xd9


 53/185 [=======>......................] - ETA: 1:00 - loss: 4.5146 - accuracy: 0.0879

Corrupt JPEG data: premature end of data segment


 95/185 [==============>...............] - ETA: 41s - loss: 4.4743 - accuracy: 0.0829

In [ ]:
test_loss, test_acc = model.evaluate(test_ds)
print(f"Test Accuracy: {test_acc * 100:.2f}%")